In [1]:
# Notebook Name: Generate_CoT_Sentiment_Analysis.ipynb
# Import necessary libraries
import os
import time
from tqdm import tqdm
from openai import OpenAI
from datasets import load_dataset, Value
from huggingface_hub import login

c:\Users\zjt20\.conda\envs\new_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
!pip install openai

# 在代码开头添加正确的导入
from openai import OpenAI 

In [1]:
import json

In [2]:
# Load API keys from Kaggle Secrets
from kaggle_secrets import UserSecretsClient


def load_kaggle_secrets():
    """
    Load API keys from Kaggle Secrets.
    """
    user_secrets = UserSecretsClient()
    api_key = user_secrets.get_secret("DeepSeek_API_KEY")
    hf_token = user_secrets.get_secret("HF_TOKEN")
    return api_key, hf_token


# Load API keys
api_key, hf_token = load_kaggle_secrets()

# Ensure API keys are available
if not api_key or not hf_token:
    raise ValueError("Missing API keys! Add 'DeepSeek_API_KEY' and 'HF_TOKEN' to your Kaggle Secrets.")

ModuleNotFoundError: No module named 'kaggle_secrets'

In [ ]:
# Function to initialize the API client
def initialize_api_client(api_key):
    """
    Initialize the OpenAI client with DeepSeek API.
    """
    return OpenAI(
        api_key=api_key,
        base_url="https://api.deepseek.com"
    )

In [ ]:
def load_data(biased_path, neutral_path):
    """加载带有详细元数据的偏见数据集"""
    from datasets import Dataset
    data = []
    
    # 加载偏见数据（包含完整元数据）
    with open(biased_path, 'r', encoding='utf-8-sig') as f:
        for item in json.load(f):
            record = {
                'original_text': item['ori_sentence'],
                'edited_text': item['edit_sentence'],
                'bias_labels': item['bias_labels'],
                'text_type': 'biased'
            }
            data.append(record)
    
    # 加载中性数据（补充必要字段）
    with open(neutral_path, 'r', encoding='utf-8-sig') as f:
        for item in json.load(f):
            record = {
                'original_text': item['text'],
                'edited_text': None,
                'bias_labels': [0, 0, 0],
                'text_type': 'neutral'
            }
            data.append(record)
    
    return Dataset.from_list(data)


In [ ]:
def generate_bias_cot(client, item, max_retries=3):
    bias_types = [
        "AC（职业活动刻板印象）",
        "DI（描述归纳偏见）", 
        "ANB（态度规范偏见）"
    ]
    
    prompt = f"""请按照以下框架分析文本的性别偏见：

【任务说明】
1. 偏见检测：判断文本是否包含性别偏见 (B/N)
2. 偏见分类：若存在偏见，按三类标注 (AC/DI/ANB)
3. 偏见缓解：生成无偏见的修改版本

【分类标准】
- AC (职业活动刻板印象): 将职业/活动与性别绑定 (例："护士应该由女性担任")
- DI (描述归纳偏见): 基于性别的特征归纳 (例："男生都不擅长文科")
- ANB (态度规范偏见): 传达性别不平等观念 (例："女孩就该文静听话")

【分析步骤】
1. 识别所有性别相关表述
2. 逐句分析偏见存在性
3. 分类标记偏见类型
4. 生成无偏见修改方案
【示例输入】
原始文本：{item["original_text"]}。
【期望输出】
{{
    "ori_sentence": {item["original_text"]},
    "is_biased": {item["text_type"]},
    "bias_labels": {item["bias_labels"]},  # AC={item["bias_labels"][0]}, DI={item["bias_labels"][1]}, ANB={item["bias_labels"][2]}
    "edit_sentence": "{item["edited_text"]}",
}}"""

    for attempt in range(max_retries):
        try:
            response = client.chat.completions.create(
                model="deepseek-chat",
                messages=[
                    {"role": "system", "content": "您是性别偏见分析专家，擅长判断性别偏见以及分类及改写性别偏见"},
                    {"role": "user", "content": prompt}
                ],
                temperature=0.4,  # 降低随机性保证分析准确性
                max_tokens=500,
                stream=False
            )
            analysis = response.choices[0].message.content.strip()
            # print(analysis)
            
            # 添加结构化结论
            conclusion_lines = [
                f"{bias_types[i]}: {label}" 
                for i, label in enumerate(item['bias_labels'])
            ]

            conclusion_lines.append(f'修改方案: {item["edited_text"]}')
            
            conclusion = "\n\n结论：\n" + "\n".join(conclusion_lines)
            
            return analysis + conclusion
        except Exception as e:
            if attempt < max_retries - 1:
                print(f"Attempt {attempt+1} failed, retrying...")
                time.sleep(2)
            else:
                print(f"Failed after {max_retries} attempts: {str(e)}")
                return "CoT generation failed"
    if "结论：" not in analysis:
        analysis += "\n结论：\n" + "\n".join(
            [f"{bias_types[i]}: {label}" for i, label in enumerate(item['bias_labels'])]
        )
    return analysis

In [ ]:
def add_cot_to_dataset(client, dataset, sample_size=1000):
    """完全正确的缩进版本"""
    # ↓ 第1级缩进（4空格）
    if sample_size and len(dataset) > sample_size:
        # ↓ 第2级缩进（8空格）
        dataset = dataset.select(range(sample_size))
    
    # ↓ 保持第1级缩进
    cots = []
    for item in tqdm(dataset):
        cot = generate_bias_cot(client, item)
        cots.append(cot)
    
    # ↓↓↓ 关键修改：统一使用Bias_Analysis_CoT作为列名 ↓↓↓
    return dataset.add_column("Bias_Analysis_CoT", cots) 



In [ ]:
biased_path = "/kaggle/input/pianjian/biased.json" 
neutral_path = "/kaggle/input/pianjian/non-biased.json"

In [ ]:
def upload_to_huggingface(dataset, repo_id, hf_token, is_private=True):
    """将数据集上传到Hugging Face Hub"""
    login(token=hf_token)
    
    # 确保字段名称与实际列名一致
    dataset = dataset.cast_column("Bias_Analysis_CoT", Value("string"))  # 修改这里
    
    dataset.push_to_hub(
        repo_id=repo_id,
        token=hf_token,
        private=is_private,
        commit_message="Add CoT annotations"
    )
    print(f"✅ 数据集已上传至: https://huggingface.co/datasets/{repo_id}")

In [ ]:
def create_cot_dataset(api_key, hf_token, dataset_name, 
                      biased_path, neutral_path, 
                      sample_size=None):  # 新增参数
    client = initialize_api_client(api_key)
    
    # 加载数据
    dataset = load_data(biased_path, neutral_path)
    
    # 传递sample_size给处理函数
    enhanced_dataset = add_cot_to_dataset(
        client=client,
        dataset=dataset,
        sample_size=sample_size  # 传递参数
    )
    
    upload_to_huggingface(enhanced_dataset, dataset_name, hf_token)
    return enhanced_dataset

In [ ]:
import os

# 检查数据集是否存在
dataset_path = "/kaggle/input/pianjian"
if os.path.exists(dataset_path):
    print("✅ 数据集目录存在")
    print("目录内容:", os.listdir(dataset_path))
else:
    print(f"❌ 路径错误: {dataset_path} 不存在")

In [ ]:
dataset_name = "pianjian/pianjian-with-cot-hou"
enhanced_dataset = create_cot_dataset(
    api_key=api_key,
    hf_token=hf_token,
    dataset_name=dataset_name,
    biased_path=biased_path,
    neutral_path=neutral_path,
    sample_size=1000 # 指定处理1000条数据
)# 修改上传函数中的列名（将Complex_CoT改为Bias_Analysis_CoT）

In [ ]:

sample_cot = enhanced_dataset[0]['Bias_Analysis_CoT']
print("\n示例CoT分析:\n", sample_cot)